In [1]:
# mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install kaggle
!pip install -q kaggle

In [3]:
# configure the kaggle API access
# create a directory for kaggle in the Colab environment
# copy the kaggle.json file from the Google Drive
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/

In [4]:
# set permission to the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# download the dataset using the kaggle API command
!kaggle datasets download tanshihjen/octdl-oct-dataset

Dataset URL: https://www.kaggle.com/datasets/tanshihjen/octdl-oct-dataset
License(s): unknown
 98% 371M/380M [00:02<00:00, 186MB/s]
100% 380M/380M [00:02<00:00, 166MB/s]


In [6]:
# unzip the download dataset
!unzip octdl-oct-dataset.zip

Archive:  octdl-oct-dataset.zip
  inflating: OCTDL/AMD/amd_1047099_1.jpg  
  inflating: OCTDL/AMD/amd_1047099_2.jpg  
  inflating: OCTDL/AMD/amd_1047099_3.jpg  
  inflating: OCTDL/AMD/amd_1047099_4.jpg  
  inflating: OCTDL/AMD/amd_1084498_1.jpg  
  inflating: OCTDL/AMD/amd_1084498_2.jpg  
  inflating: OCTDL/AMD/amd_1084498_3.jpg  
  inflating: OCTDL/AMD/amd_1096590_1.jpg  
  inflating: OCTDL/AMD/amd_1096590_2.jpg  
  inflating: OCTDL/AMD/amd_1096590_3.jpg  
  inflating: OCTDL/AMD/amd_1096590_4.jpg  
  inflating: OCTDL/AMD/amd_1096590_5.jpg  
  inflating: OCTDL/AMD/amd_1101816_1.jpg  
  inflating: OCTDL/AMD/amd_1101816_2.jpg  
  inflating: OCTDL/AMD/amd_1101816_3.jpg  
  inflating: OCTDL/AMD/amd_1101816_4.jpg  
  inflating: OCTDL/AMD/amd_1131743_1.jpg  
  inflating: OCTDL/AMD/amd_1163930_1.jpg  
  inflating: OCTDL/AMD/amd_1163930_2.jpg  
  inflating: OCTDL/AMD/amd_1163930_3.jpg  
  inflating: OCTDL/AMD/amd_1168623_1.jpg  
  inflating: OCTDL/AMD/amd_1168623_2.jpg  
  inflating: OCTDL/AMD

In [7]:
# import essential libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow import keras
from keras import models, layers

In [ ]:
# load the training and validation dataset
train_df = pd.read_csv('/content/drive/MyDrive/dataset/oct_train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/dataset/oct_val.csv')

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    preprocessing_function = None,
    rotation_range = 0.2,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.0,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest',
)

val_datagen = ImageDataGenerator(preprocessing_function=None)

In [27]:
# prepare the dataset
train_ds = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = None,
    x_col = 'Path',
    y_col = 'Label',
    class_mode = 'categorical',
    batch_size = 32,
    target_size = (224, 224),
    shuffle = True,
    seed = 122,
)

val_ds = val_datagen.flow_from_dataframe(
    dataframe = val_df,
    directory = None,
    x_col = 'Path',
    y_col = 'Label',
    class_mode = 'categorical',
    batch_size = 32,
    target_size = (224, 224),
    shuffle = False
)

Found 1454 validated image filenames belonging to 7 classes.
Found 203 validated image filenames belonging to 7 classes.


In [29]:
# define list of callbacks to enhance model training
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/models/oct_enetv2m_sp_151124.keras', save_best_only=True),
    keras.callbacks.CSVLogger(filename='/content/drive/MyDrive/learning_history/oct_enetv2m_sp_151124.csv'),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='min', min_lr=0.00001)
    ]

In [30]:
# metrics to be used for compiling the model
metrics = [
      tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc')
]

In [31]:
from keras.applications import EfficientNetV2M

# helper function to define the neural network on top of the pre-trained model
def tf_model():
  input_shape = (224, 224, 3)

  inputs = keras.Input(shape=input_shape)
  x = tf.keras.layers.Rescaling(1./255)(inputs)

  base_model = EfficientNetV2M(
    include_top = False,
    weights = 'imagenet',
    input_shape = input_shape,
    pooling = 'avg'
)

  base_model.trainable = True

  x = base_model(x)

  x = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
  x = layers.Dense(units=256, activation='relu')(x)
  x = layers.Dense(units=128, activation='relu')(x)
  x = layers.Dropout(rate=0.25, seed=122)(x)
  output = layers.Dense(units=7, activation='softmax')(x)

  model = keras.models.Model(inputs=inputs, outputs=output)

  model.compile(optimizer = keras.optimizers.Adamax(learning_rate=0.001),
                loss = keras.losses.categorical_crossentropy,
                metrics = metrics
                )

  return model

In [32]:
enetv2m_sp_151124 = tf_model()

In [33]:
history = enetv2m_sp_151124.fit(train_ds,
                                epochs = 100,
                                validation_data = val_ds,
                                callbacks = callbacks)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


46/46 ━━━━━━━━━━━━━━━━━━━━ 393s 4s/step - auc: 0.7852 - categorical_accuracy: 0.5257 - loss: 1.6357 - precision: 0.6216 - recall: 0.4115 - val_auc: 0.9074 - val_categorical_accuracy: 0.6552 - val_loss: 1.6182 - val_precision: 1.0000 - val_recall: 0.0049 - learning_rate: 0.0010
Epoch 2/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 37s 725ms/step - auc: 0.9248 - categorical_accuracy: 0.6891 - loss: 0.9436 - precision: 0.8089 - recall: 0.5991 - val_auc: 0.9088 - val_categorical_accuracy: 0.6798 - val_loss: 1.4318 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 37s 723ms/step - auc: 0.9553 - categorical_accuracy: 0.7632 - loss: 0.7178 - precision: 0.8459 - recall: 0.6657 - val_auc: 0.9767 - val_categorical_accuracy: 0.8473 - val_loss: 0.6843 - val_precision: 0.9704 - val_recall: 0.6453 - learning_rate: 0.0010
Epoch 4/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 37s 728ms/step - auc: 0.9752 - categorical_accuracy: 0.8151 - loss: 0.5242 - precision: 0.8

In [34]:
enetv2m_sp_151124.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling_2 (Rescaling)              │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetv2-m (Functional)        │ (None, 1280)                │      53,150,388 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 1280)                │           5,120 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 159,962,547 (610.21 MB)

 Trainable params: 53,222,651 (203.03 MB)

 Non-trainable params: 294,592 (1.12 MB)

 Optimizer params: 106,445,304 (406.06 MB)

In [35]:
# export the trained model
enetv2m_sp_151124.save('/content/drive/MyDrive/models/enetv2m_sp_151124.h5')